<a href="https://colab.research.google.com/github/PnrSvc/Bert-Examples/blob/main/bert-example_tr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline
import torch
import requests
from bs4 import BeautifulSoup
import re 
import numpy as np
import pandas as pd

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.5 MB/s eta 0:11:17tcmalloc: large alloc 1147494400 bytes == 0x5642d7b38000 @  0x7f84699ce615 0x56429f1673bc 0x56429f24818a 0x56429f16a1cd 0x56429f25cb3d 0x56429f1de458 0x56429f1d902f 0x56429f16baba 0x56429f1de2c0 0x56429f1d902f 0x56429f16baba 0x56429f1dacd4 0x56429f25d986 0x56429f1da350 0x56429f25d986 0x56429f1da350 0x56429f25d986 0x56429f1da350 0x56429f16bf19 0x56429f1afa79 0x56429f16ab32 0x56429f1de1dd 0x56429f1d902f 0x56429f16baba 0x56429f1dacd4 0x56429f1d902f 0x56429f16baba 0x56429f1d9eae 0x56429f16b9da 0x56429f1da108 0x56429f1d902f
     |██████████████████▌             | 1055.7 MB 1.6 MB/s eta 0:07:59tcmalloc: large alloc 1434370048 bytes == 0x56431c18e000 @  0x7f84699ce615 0x56429f1673bc 0x56429f24818a 0x56429f16a1cd 0x56429f25cb3d 0x56429f1de458 0x56429f1d902f 0x56429f16baba 0x56429f1de2c0 0x56429f1d902f 0x56429f16baba 0x56429f1dacd4 0x56429f25d986 0x5642

In [2]:
tokenizer=AutoTokenizer.from_pretrained('savasy/bert-base-turkish-sentiment-cased')
model=AutoModelForSequenceClassification.from_pretrained('savasy/bert-base-turkish-sentiment-cased')
turk=pipeline("sentiment-analysis",tokenizer=tokenizer,model=model)

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/596 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [3]:
r=requests.get('https://www.gittigidiyor.com/apple-iphone-13/yorumlari')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*user-catalog-review-comment-detail gg-d-23 gg-m-22 pl0.*')
result=soup.find_all('div',{'class':regex})
reviews=[result.text.strip() for result in result]

In [4]:
reviews

['Eşimin doğum günü hediyesi için geçen günlerde satın almıştım. Telefon geldiğinden beri elinden düşürmedi. Kameranın kalitesinden dolayı sürekli fotoğraf ve video çekiyor. Yüksek saklama alanı  her uygulamayı rahatça indirebileceği alanı sunuyor. Tasarımı ekstra şık. Apple bu modelinde batarya ömrünü de uzatmış. 2,5 saat kadar daha uzun süre kullanabiliyorsunuz. Tüm özellikleri ile bir tasarım harikası bana kalırsa. Kesinlikle tavsiye ederim.',
 'Harika, telefonu elinden düşürmeyen biri olarak çok iyi derece de şarjı gidiyor.',
 'Telefonumu yükseltmek istediğim için geçtiğim hafta bu telefonu satın aldım. Apple kalitesi beni yine şaşırtmadı. Pil ömrü çok dayanıklı, gün içinde şarj etmeden rahatça kullanabiliyorum. Tasarımı çok şık ve telefonun malzemesi oldukça kaliteli. Suya dayanıklılığı sayesinde havuzda korkmadan rahatça kullanabiliyorum. Gelişmiş özellikleri ile bu telefona sahip olduğunuz için kendinizi ayrıcalıklı hissedeceğinizden hiç şüpheniz olmasın.',
 'Kalite tesadüf deği

In [7]:
df=pd.DataFrame(np.array(reviews), columns=['review'])

In [8]:
import nltk
nltk.download('punkt')
df["tokenized_sents"] = df.apply(lambda row: nltk.sent_tokenize(row["review"]), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
df.head(3)

,review,tokenized_sents
0,Eşimin doğum günü hediyesi için geçen günlerde...,[Eşimin doğum günü hediyesi için geçen günlerd...
1,"Harika, telefonu elinden düşürmeyen biri olara...","[Harika, telefonu elinden düşürmeyen biri olar..."
2,Telefonumu yükseltmek istediğim için geçtiğim ...,[Telefonumu yükseltmek istediğim için geçtiğim...


In [10]:
for x in df['tokenized_sents']:
  p=turk(x)
  print(p)

[{'label': 'negative', 'score': 0.9707889556884766}, {'label': 'positive', 'score': 0.9839617013931274}, {'label': 'positive', 'score': 0.9877925515174866}, {'label': 'positive', 'score': 0.5978514552116394}, {'label': 'negative', 'score': 0.8675971031188965}, {'label': 'negative', 'score': 0.9991732239723206}, {'label': 'negative', 'score': 0.9873930811882019}, {'label': 'positive', 'score': 0.9908479452133179}, {'label': 'positive', 'score': 0.9891648888587952}]
[{'label': 'positive', 'score': 0.6274456977844238}]
[{'label': 'negative', 'score': 0.9985653758049011}, {'label': 'negative', 'score': 0.9988102912902832}, {'label': 'negative', 'score': 0.9834717512130737}, {'label': 'positive', 'score': 0.5972719192504883}, {'label': 'negative', 'score': 0.7363184690475464}, {'label': 'positive', 'score': 0.6475696563720703}]
[{'label': 'positive', 'score': 0.9978017210960388}]
[{'label': 'positive', 'score': 0.7007154226303101}]
[{'label': 'positive', 'score': 0.7007154226303101}]
[{'lab

In [11]:
df['tokenized_sents'].iloc[1]

['Harika, telefonu elinden düşürmeyen biri olarak çok iyi derece de şarjı gidiyor.']

In [12]:
P=turk('ürün muhteşem')
print(P)

[{'label': 'positive', 'score': 0.9904757142066956}]
